In [3]:
import os
import logging

import mrcfile
import numpy as np
import matplotlib.pyplot as plt

import cl_utils as cl

from scipy import ndimage as ndi
from scipy.spatial.transform import Rotation as R

from aspire.image import Image
from aspire.noise import WhiteNoiseAdder
from aspire.operators import ScalarFilter
from aspire.source.simulation import Simulation
from aspire.source import ArrayImageSource
from aspire.utils import Rotation, gaussian_2d, rots_to_clmatrix
from aspire.volume import Volume
from aspire.abinitio import CLSyncVoting

ModuleNotFoundError: No module named 'aspire'

In [2]:
### some helper functions

def rot_matrix_2d(theta):
    return np.array([[np.cos(theta), -np.sin(theta)],
                     [np.sin(theta),  np.cos(theta)]])


def form_cl_matrix_from_cl_angles(cl_angles, n_total):
    
    A = np.zeros((2*n_total, n_total))

    point = np.array([[1, 0]]).T

    for pair, angles in cl_angles.items():
        i, j = pair

        theta_ij = np.deg2rad(angles[0])
        theta_ji = np.deg2rad(angles[1])

        Ri = rot_matrix_2d(theta_ij)
        Rj = rot_matrix_2d(theta_ji)

        ri = Ri @ point
        rj = Rj @ point

        A[2*i:2*i+2, j] = ri.T  
        A[2*j:2*j+2, i] = rj.T
        
    return A


def within_n_degrees(gt_clmatrix, est_clmatrix, n):
    """number images within n_degrees of gt common line angle"""
    
    angle_diffs = abs(est_clmatrix - gt_clmatrix)
    within_n = np.count_nonzero(angle_diffs <= n)
    within_n += np.count_nonzero(angle_diffs >= 360-n)
    
    return within_n / angle_diffs.size

In [3]:
### Inputs ###
np.random.seed(0)

DATA_DIR = '/scratch/network/ev9102/Data/'  # set the input directory with your EMDBs 
emd_id = 'emd_2858.map'  # 40S, 60S, 80S, Apo = 'emd_22052.map'
n_img = 10  # number of images for each structure
L = 128  # size to downsample the volume
snr = 1  # snr for additive wgn to projection images
add_mask = True  # add circular mask to images
mask_radius = L//2  # in pixels
use_noisy = False  # option to use the noisy images of clean images
angles = np.arange(0, 360, 1)  # angular sampling for line projections
r = cl.get_random_euler_angles(n_img) # pick Euler angles (in rads) for 'ZYZ' convention

In [4]:
logger = logging.getLogger(__name__)

with mrcfile.open(os.path.join(DATA_DIR, emd_id)) as mrc:
    volume = mrc.data
    vox = mrc.voxel_size.x
    mrc.close()

volume = Volume(volume).downsample(L)

In [ ]:
rots = Rotation.from_euler(r)  # list of rotations in Aspire format
shifts = np.zeros((n_img, 2))  # not including shifts for now
amplitudes = np.ones(n_img)

src = Simulation(vols=volume,  # our Volume
                  L=volume.resolution,  # resolution, should match Volume
                  n=n_img,  # number of projection images
                  angles=rots.angles,  # pass our rotations as Euler angles
                  offsets=shifts,  # translations (wrt to origin)
                  amplitudes=amplitudes,  # amplification ( 1 is identity)
                  noise_adder=WhiteNoiseAdder.from_snr(snr=snr),
                  seed=12345,  # RNG seed for reproducibility
                  dtype=volume.dtype  # match our datatype to the Volume.
                )

clean_images = src.projections[:].asnumpy()
noisy_images = src.images[:].asnumpy()

if use_noisy:
    images = noisy_images
else:
    images = clean_images

2024-02-22 15:15:36,959 INFO [aspire.source.image] Creating Simulation with 10 images.
2024-02-22 15:15:36,965 INFO [aspire.source.simulation] Appending WhiteNoiseAdder with variance=None to generation pipeline


100%|██████████| 1/1 [00:00<00:00,  1.03it/s]


In [ ]:
# ### compute the line projection distances and form common lines matrix
# ### format is (img1, img2) : (theta1, theta2) --> angles to rotate for common lines using np conventions
# cl_angles = cl.compute_common_line_angles(images, angles)

# ### now form A, the common lines matrix
# A = cl.form_cl_matrix_from_cl_angles(cl_angles, n_img)

In [ ]:
### etsimate common lines and common lines matrix using aspire

aspire_images = ArrayImageSource(images)  # pass images back to aspire class

orient_est = CLSyncVoting(aspire_images, n_theta=360, max_shift=0.15, shift_step=0.5)
orient_est.build_clmatrix()

est_cl_matrix = orient_est.clmatrix
gt_cl_matrix = rots_to_clmatrix(src.rotations, n_theta=360)

cl_angles = {}

for i in range(n_img-1):
    for j in range(i+1, n_img):
        cl_angles[(i, j)] = [est_cl_matrix[i, j], est_cl_matrix[j, i]]
        
A = form_cl_matrix_from_cl_angles(cl_angles, n_img)

plt.imshow(est_cl_matrix)
plt.colorbar()
plt.show()
plt.imshow(A)
plt.colorbar()
plt.show()

In [ ]:
### Visualize a common line projection

idx_i = 0
idx_j = 1

theta_i = cl_angles[(idx_i, idx_j)][0]
theta_j = cl_angles[(idx_i, idx_j)][1]

img_i = ndi.rotate(images[idx_i], theta_i, reshape=False)
img_j = ndi.rotate(images[idx_j], theta_j, reshape=False)

c_ij = np.sum(img_i, axis=0)
c_ji = np.sum(img_j, axis=0)

cl.two_plot(img_i, img_j)
plt.plot(c_ij)
plt.plot(c_ji)
plt.show()

In [ ]:
print(within_n_degrees(gt_cl_matrix, est_cl_matrix, n=5))

In [ ]:
# ### save data to matlab format
# from scipy.io import savemat

# est_cl_mat_dict = {'est_cl_matrix': est_cl_matrix}
# savemat('Data/updated_aspire_estimated_cl_matrix.m', est_cl_mat_dict)

# gt_cl_mat_dict = {'gt_cl_matrix': gt_cl_matrix}
# savemat('Data/updated_aspire_groundtruth_cl_matrix.m', gt_cl_matrix_dict)

# gt_rotations_dict = {'gt_rotations': rots.matrices}
# savemat('Data/updated_aspire_rotation_matrix.m', gt_rotations_dict)